# Steady State Heat Conduction Through Multiple Domains

This notebook explains how to set up a simple problem with several connected domains using steady state thermal conduction through three connected domains as an example. 

![Problem Sketch](./heat_conduction_sketch.svg "Problem Sketch")

To do this, the heat equation is solved for each domain:

$$
k \left(\frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2} \right) = 0 
$$

Several boundary conditions need to be satisfied:
 * An adiabatic boundary at x=0 is defined as: $\frac{\partial T_{x=0}}{\partial x} = 0$
 * An isothermal boundary setting y=0 to 100C is defined as: $T_{y=0} = 100$
 * To connect two boundaries, temperature and flux are equal: $Ta_{x=-1} -Tb_{x=0} = 0$ and $\frac{\partial Ta_{x=-1}}{\partial x} - \frac{\partial Tb_{x=0}}{\partial x} = 0$

In [ ]:
import fastfd as ffd
ffd.sparse_lib('scipy')

In [ ]:
# Define linear axes to set grid resolution in each dimension of the model
x_a = ffd.LinearAxis('x', start = 0, stop = 0.5, num = 51)
x_b = ffd.LinearAxis('x', start = 0.5, stop = 0.7, num = 21)
x_c = ffd.LinearAxis('x', start = 0.7, stop = 1.2, num = 51)
y = ffd.LinearAxis('y', start = 0, stop = 1, num = 101)

# Define scalars
# Axis names must be unique to each Scalar, but can be reused in different scalars.
# Default approximation accuracy can be set at the scalar level, or for individual derivitive calls
T_a = ffd.Scalar('T_a', [x_a, y], accuracy = 4)
T_b = ffd.Scalar('T_b', [x_b, y], accuracy = 4)
T_c = ffd.Scalar('T_c', [x_c, y], accuracy = 4)

model = ffd.FDModel([T_a, T_b, T_c])

In [ ]:
# Thermal Conductivity of domains a, b, and c
k_a = 3
k_b = 0.5
k_c = 1

# Set model governing equations (thermal diffusion)
# Equations are specied as a dictionary:
#     {'Key', (coefficient matrix, constraint vector)}
model.update_equations({
    'Cond A': (k_a * (T_a.d('x', 2) + T_a.d('y', 2)),
               0),
    'Cond B': (k_b * (T_b.d('x', 2) + T_b.d('y', 2)),
               0),
    'Cond C': (k_c * (T_c.d('x', 2) + T_c.d('y', 2)),
               0),
})

# Set model boundary conditions
# Boundary conditions are specied as a dictionary with format:
#     {'Key', (mask, coefficient matrix, constraint vector)}
    
# Masks must be Scalar identites. Slices can be used selectively apply boundary conditions. Each
# mask and coefficient matrix must have the same shape.
model.update_bocos({
    'Ta(x=0) adiabatic': (T_a.i[0, :], T_a.d('x')[0, :], 0),
    'Ta(y=-1) adiabatic': (T_a.i[:, -1], T_a.d('y')[:, -1], 0),
    'Ta(y=0) temp': (T_a.i[:, 0], T_a.i[:, 0], 100),
    
    'Tb(y=0) adiabatic': (T_b.i[:, 0], T_b.d('y')[:, 0], 0),
    'Tb(y=-1) adiabatic': (T_b.i[:, -1], T_b.d('y')[:, -1], 0),
    
    'Tc(x=-1) adiabatic': (T_c.i[-1, :], T_c.d('x')[-1, :], 0),
    'Tc(y=0) adiabatic': (T_c.i[:, 0], T_c.d('y')[:, 0], 0),
    'Tc(y=-1) temp': (T_c.i[:, -1], T_c.i[:, -1], 200),
    
    '=flux Ta(x=-1) Tb(x=0)': (T_a.i[-1, :], k_a * T_a.d('x')[-1, :] - k_b * T_b.d('x')[0, :], 0),
    '= Ta(x=-1) Tb(x=0)': (T_b.i[0, :], T_a.i[-1, :] - T_b.i[0, :], 0),
    
    '=flux Tb(x=-1) Tc(x=0)': (T_c.i[0, :], k_b * T_b.d('x')[-1, :] - k_c * T_c.d('x')[0, :], 0),
    '= Tb(x=-1) Tc(x=0)': (T_b.i[-1, :], T_b.i[-1, :] - T_c.i[0, :], 0),
})

# Solve the model
result = model.solve()

In [ ]:
# Plot Results
import numpy as np
import holoviews as hv
hv.extension('bokeh')

Image = hv.Image(
    np.hstack([result['T_a'].T, result['T_b'][1:-1,:].T, result['T_c'].T])[::-1,:],
    bounds = (0, 0, 1.2, 1)
).opts(
    cmap = 'CMRmap',
    colorbar = True)

hv.Overlay([
    Image,
    hv.operation.contours(Image, levels = 15).opts(cmap = 'gray_r'),
    hv.Curve(((0.5, 0.5), (0, 1))).opts(color = 'k'),
    hv.Curve(((0.7, 0.7), (0, 1))).opts(color = 'k'),
]).opts(
    width = 700, height = 600,
    show_legend = False
)